# Open-Source GIS Programming in Python: A Hands-On Workshop

**Instructor:** Harvey Jing  
**Duration:** 3 Hours  
**Format:** Hands-on Lab Session

**Workshop Goal:** How much of the population and how many buildings are exposed to wildfire risk in Los Angeles County?

---

## Workshop Overview

This workshop will demonstrate the power of open-source GIS programming using Python libraries:
- **GeoPandas** - Vector data manipulation
- **Rasterio** - Raster data processing  
- **Rasterstats** - Spatial statistics
- **Leafmap** - Interactive mapping and visualization

We'll work through a complete geospatial analysis workflow to answer our research question about wildfire exposure in LA County.

---

## Module 0: Introduction & Setup (20 minutes)

### Environment Setup
Let's start by importing the necessary libraries and setting up our workspace.

In [ ]:
# Import core libraries for geospatial analysis
import geopandas as gpd
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterstats
import leafmap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up data paths (using pseudo filenames as requested)
DATA_DIR = Path("data/")
VECTOR_DIR = DATA_DIR / "vector"
RASTER_DIR = DATA_DIR / "raster"

print("Libraries imported successfully!")
print(f"GeoPandas version: {gpd.__version__}")
print(f"Rasterio version: {rasterio.__version__}")
print(f"Leafmap version: {leafmap.__version__}")
print("Environment setup complete!")

---

## Module 1: A Quick Look with a GenAI Co-Pilot (25 minutes)

### Quick-Start Demo
Let's start with a high-level overview by creating a simple visualization that combines vector and raster data. This will give you a preview of what we'll accomplish by the end of the workshop.

In [ ]:
# Quick demo: Load and visualize LA County boundary and wildfire hazard data
# Note: Using pseudo filenames - replace with actual file paths

# 1. Read LA County boundary (vector data)
la_county_file = VECTOR_DIR / "la_county_boundary.shp" 
# la_county = gpd.read_file(la_county_file)

# 2. Read Wildfire Hazard Potential raster
whp_raster_file = RASTER_DIR / "wildfire_hazard_potential.tif"
# whp_raster = rasterio.open(whp_raster_file)

# 3. Create an interactive map combining both layers
# This is what we'll build step by step throughout the workshop!

print("Quick demo setup complete!")
print("Files to be loaded:")
print(f"- LA County boundary: {la_county_file}")
print(f"- Wildfire Hazard Potential: {whp_raster_file}")
print("\nBy the end of this workshop, you'll have:")
print("- Loaded and processed vector and raster data")
print("- Calculated population and building exposure to wildfire risk")
print("- Created interactive maps to visualize the results")

### Using Generative AI as Your Programming Assistant

Throughout this workshop, you can use AI tools to:
- Generate boilerplate code
- Explain complex functions
- Debug errors
- Suggest alternative approaches

**Pro tip:** Try asking an AI assistant to explain any code sections you find confusing!

---

## Module 2: Vector Data Wrangling with GeoPandas (45 minutes)

In this section, we'll prepare our socioeconomic and infrastructure layers using the following datasets:
- LA County Boundary
- California Census Tracts (with population data)
- California Building Footprints

### Reading and Understanding Geospatial Vector Data

In [ ]:
# Load vector datasets from various sources

# 1. Load LA County boundary from Shapefile
la_county_file = VECTOR_DIR / "la_county_boundary.shp"
# la_county = gpd.read_file(la_county_file)

# 2. Load California Census Tracts with population data
census_tracts_file = VECTOR_DIR / "ca_census_tracts_with_population.shp" 
# census_tracts = gpd.read_file(census_tracts_file)

# 3. Load California Building Footprints
buildings_file = VECTOR_DIR / "ca_building_footprints.shp"
# buildings = gpd.read_file(buildings_file)

# Alternative: Load from spatial database (DuckDB example)
# import duckdb
# conn = duckdb.connect()
# conn.execute("INSTALL spatial")
# conn.execute("LOAD spatial")
# la_county = conn.execute("SELECT * FROM ST_Read('la_county_boundary.shp')").df()

print("Vector data loading setup complete!")
print("Datasets to be loaded:")
print(f"- LA County boundary: {la_county_file}")
print(f"- Census tracts: {census_tracts_file}") 
print(f"- Building footprints: {buildings_file}")

# Understanding GeoDataFrame structure
print("\nGeoDataFrame characteristics:")
print("- Similar to pandas DataFrame")
print("- Contains a special 'geometry' column with spatial data")
print("- Supports spatial operations and analysis")

### Selecting Data - Attribute-based and Location-based Selection

In [ ]:
# Attribute-based selection: Filter census tracts for LA County
# Method 1: Using GEOID (if available)
# la_census_tracts = census_tracts[census_tracts['GEOID'].str.startswith('06037')]

# Method 2: Using County Name (if available)
# la_census_tracts = census_tracts[census_tracts['COUNTY_NAME'] == 'Los Angeles County']

print("Attribute-based selection examples:")
print("# Filter by GEOID:")
print("la_census_tracts = census_tracts[census_tracts['GEOID'].str.startswith('06037')]")
print("# Filter by County Name:")
print("la_census_tracts = census_tracts[census_tracts['COUNTY_NAME'] == 'Los Angeles County']")

# Location-based selection: Spatial operations
print("\nLocation-based selection examples:")
print("# Spatial intersection (clip):")
print("la_census_tracts = gpd.overlay(census_tracts, la_county, how='intersection')")
print("# Spatial join (select features within boundary):")
print("la_buildings = gpd.sjoin(buildings, la_county, how='inner', predicate='within')")

# Check coordinate reference systems
print("\nImportant: Always check and align CRS!")
print("print(la_county.crs)")
print("print(census_tracts.crs)")
print("# Reproject if needed:")
print("census_tracts = census_tracts.to_crs(la_county.crs)")

### Data Export and Visualization

In [ ]:
# Export cleaned data to GeoPackage format
output_dir = Path("output/")
# output_dir.mkdir(exist_ok=True)

# la_county.to_file(output_dir / "la_county_clean.gpkg", layer="county_boundary", driver="GPKG")
# la_census_tracts.to_file(output_dir / "la_county_clean.gpkg", layer="census_tracts", driver="GPKG")
# la_buildings.to_file(output_dir / "la_county_clean.gpkg", layer="buildings", driver="GPKG")

print("Data export commands:")
print("# Save to GeoPackage (recommended for multiple layers):")
print("la_county.to_file('output/la_county_clean.gpkg', layer='county_boundary')")
print("la_census_tracts.to_file('output/la_county_clean.gpkg', layer='census_tracts')")
print("la_buildings.to_file('output/la_county_clean.gpkg', layer='buildings')")

# Basic visualization with leafmap
print("\nVisualization with leafmap:")
print("m = leafmap.Map(center=[34.0522, -118.2437], zoom=8)")
print("m.add_gdf(la_county, layer_name='LA County Boundary')")
print("m.add_gdf(la_census_tracts, layer_name='Census Tracts')")
print("m")

# Alternative: Quick plot with matplotlib
print("\nAlternative: Quick static plot:")
print("fig, ax = plt.subplots(figsize=(10, 8))")
print("la_county.plot(ax=ax, color='lightblue', edgecolor='black')")
print("la_census_tracts.plot(ax=ax, color='orange', alpha=0.5)")
print("plt.title('LA County Census Tracts')")
print("plt.show()")

---

## Module 3: Raster Data Processing with Rasterio (30 minutes)

Now we'll prepare our wildfire hazard layer using the Wildfire Hazard Potential (WHP) raster dataset. Classes 1-5 represent risk levels from very low to very high.

### Reading and Understanding Geospatial Raster Data

In [ ]:
# Open and explore the WHP raster file
whp_file = RASTER_DIR / "wildfire_hazard_potential.tif"

# with rasterio.open(whp_file) as whp_raster:
#     # Read basic information
#     print(f"Driver: {whp_raster.driver}")
#     print(f"Width: {whp_raster.width}")
#     print(f"Height: {whp_raster.height}")
#     print(f"Number of bands: {whp_raster.count}")
#     print(f"Data type: {whp_raster.dtypes[0]}")
#     print(f"CRS: {whp_raster.crs}")
#     print(f"Transform: {whp_raster.transform}")
#     print(f"Bounds: {whp_raster.bounds}")
    
#     # Read the data
#     whp_data = whp_raster.read(1)  # Read first (and only) band
#     print(f"Data shape: {whp_data.shape}")
#     print(f"Unique values: {np.unique(whp_data)}")

print("Raster data exploration commands:")
print("with rasterio.open('wildfire_hazard_potential.tif') as src:")
print("    print(f'CRS: {src.crs}')")
print("    print(f'Transform: {src.transform}')")
print("    print(f'Shape: {src.width} x {src.height}')")
print("    data = src.read(1)  # Read first band")
print("    print(f'Unique values: {np.unique(data)}')")

print("\nWHP Risk Classes:")
print("1 = Very Low Risk")
print("2 = Low Risk") 
print("3 = Moderate Risk")
print("4 = High Risk")
print("5 = Very High Risk")

### Preparing the Raster for Analysis

In [ ]:
# Reproject raster if necessary to match vector data CRS
def reproject_raster(src_path, dst_path, dst_crs):
    """Reproject raster to target CRS"""
    # with rasterio.open(src_path) as src:
    #     transform, width, height = calculate_default_transform(
    #         src.crs, dst_crs, src.width, src.height, *src.bounds)
    #     kwargs = src.meta.copy()
    #     kwargs.update({
    #         'crs': dst_crs,
    #         'transform': transform,
    #         'width': width,
    #         'height': height
    #     })
    #     
    #     with rasterio.open(dst_path, 'w', **kwargs) as dst:
    #         reproject(
    #             source=rasterio.band(src, 1),
    #             destination=rasterio.band(dst, 1),
    #             src_transform=src.transform,
    #             src_crs=src.crs,
    #             dst_transform=transform,
    #             dst_crs=dst_crs,
    #             resampling=Resampling.nearest)
    pass

# Clip raster to LA County boundary
def clip_raster_to_boundary(raster_path, boundary_gdf, output_path):
    """Clip raster to vector boundary"""
    # with rasterio.open(raster_path) as src:
    #     # Convert boundary to same CRS as raster
    #     boundary_reproj = boundary_gdf.to_crs(src.crs)
    #     
    #     # Get geometry in GeoJSON format
    #     geoms = [mapping(geom) for geom in boundary_reproj.geometry]
    #     
    #     # Clip the raster
    #     out_image, out_transform = mask(src, geoms, crop=True)
    #     out_meta = src.meta.copy()
    #     out_meta.update({
    #         "driver": "GTiff",
    #         "height": out_image.shape[1],
    #         "width": out_image.shape[2],
    #         "transform": out_transform
    #     })
    #     
    #     with rasterio.open(output_path, "w", **out_meta) as dest:
    #         dest.write(out_image)
    pass

print("Raster processing functions defined:")
print("1. reproject_raster() - Reprojects raster to match vector CRS")
print("2. clip_raster_to_boundary() - Clips raster to LA County boundary")
print("\nExample usage:")
print("# Reproject if needed:")
print("reproject_raster('whp_original.tif', 'whp_reprojected.tif', la_county.crs)")
print("# Clip to LA County:")
print("clip_raster_to_boundary('whp_reprojected.tif', la_county, 'whp_la_county.tif')")

---

## Module 4: Spatial Analysis - Quantifying Wildfire Exposure (45 minutes)

This is where we combine our vector and raster layers to answer our research question. We'll use the **rasterstats** library for efficient zonal statistics.

### 1. Calculating Population Exposure

In [ ]:
# Calculate population exposure using zonal statistics

# Goal: Estimate population living in high-risk wildfire areas for each census tract

# Using rasterstats to calculate pixel counts for each WHP risk category
whp_clipped = RASTER_DIR / "whp_la_county.tif"

# Method 1: Count pixels in each risk category
# whp_stats = rasterstats.zonal_stats(
#     la_census_tracts, 
#     whp_clipped, 
#     categorical=True,
#     category_map={1: 'very_low', 2: 'low', 3: 'moderate', 4: 'high', 5: 'very_high'}
# )

# Convert results to DataFrame and join back to census tracts
# whp_df = pd.DataFrame(whp_stats)
# la_census_tracts_exposure = la_census_tracts.join(whp_df)

# Calculate population at risk (assuming population is evenly distributed)
# la_census_tracts_exposure['pop_high_risk'] = (
#     la_census_tracts_exposure['POPULATION'] * 
#     (la_census_tracts_exposure['high'] + la_census_tracts_exposure['very_high']) / 
#     (la_census_tracts_exposure['very_low'] + la_census_tracts_exposure['low'] + 
#      la_census_tracts_exposure['moderate'] + la_census_tracts_exposure['high'] + 
#      la_census_tracts_exposure['very_high'])
# )

print("Population exposure calculation:")
print("1. Use rasterstats.zonal_stats() with categorical=True")
print("2. Count pixels in each WHP risk category per census tract")
print("3. Calculate proportion of high-risk pixels")
print("4. Multiply by total population to estimate exposed population")

print("\nExample code:")
print("whp_stats = rasterstats.zonal_stats(census_tracts, whp_raster, categorical=True)")
print("census_tracts['pop_high_risk'] = census_tracts['POPULATION'] * high_risk_proportion")

### 2. Calculating Building Exposure

In [ ]:
# Calculate building exposure to wildfire risk

# Goal: Determine number of buildings exposed to wildfire risk

# Method 1: Spatial join to identify buildings in high-risk areas
# First, create high-risk zones (WHP classes 4 and 5)
# high_risk_mask = (whp_data >= 4)

# Method 2: Direct spatial join with buildings
# buildings_with_risk = gpd.sjoin(la_buildings, la_census_tracts_exposure, how='left')

# Calculate percentage of exposed buildings per census tract
def calculate_building_exposure(buildings_gdf, census_tracts_gdf):
    """Calculate percentage of buildings exposed to wildfire per census tract"""
    # Spatial join buildings with census tracts
    # buildings_with_tracts = gpd.sjoin(buildings_gdf, census_tracts_gdf, how='left')
    
    # Group by census tract and calculate exposure metrics
    # exposure_stats = buildings_with_tracts.groupby('index_right').agg({
    #     'OBJECTID': 'count',  # Total buildings
    #     'high_risk_flag': 'sum'  # Buildings in high risk areas
    # }).rename(columns={'OBJECTID': 'total_buildings', 'high_risk_flag': 'exposed_buildings'})
    
    # Calculate percentage
    # exposure_stats['exposure_percentage'] = (
    #     exposure_stats['exposed_buildings'] / exposure_stats['total_buildings'] * 100
    # )
    
    # return exposure_stats
    pass

print("Building exposure calculation methods:")
print("1. Spatial join buildings with high-risk wildfire zones")
print("2. Calculate percentage: (exposed buildings / total buildings) * 100")
print("3. Aggregate results by census tract")

print("\nExample workflow:")
print("# Create high-risk mask")
print("high_risk_zones = rasterize_high_risk_areas(whp_raster)")
print("# Spatial join")
print("exposed_buildings = gpd.sjoin(buildings, high_risk_zones)")
print("# Calculate percentages by census tract")
print("exposure_stats = calculate_exposure_percentage(exposed_buildings)")

### 3. Quantifying Building-Level Risk

In [ ]:
# Assign specific WHP risk class to each building

# Goal: Assign WHP risk class to each individual building footprint

# Use zonal statistics to find majority (mode) WHP class within each building
def assign_building_risk_class(buildings_gdf, whp_raster_path):
    """Assign WHP risk class to each building using majority rule"""
    
    # Calculate zonal statistics for each building footprint
    # building_stats = rasterstats.zonal_stats(
    #     buildings_gdf,
    #     whp_raster_path,
    #     stats=['majority'],  # Get the most common value
    #     nodata=-9999
    # )
    
    # Convert to DataFrame and merge with buildings
    # risk_df = pd.DataFrame(building_stats)
    # buildings_with_risk = buildings_gdf.copy()
    # buildings_with_risk['whp_risk_class'] = risk_df['majority']
    
    # Create risk category labels
    # risk_labels = {1: 'Very Low', 2: 'Low', 3: 'Moderate', 4: 'High', 5: 'Very High'}
    # buildings_with_risk['risk_category'] = buildings_with_risk['whp_risk_class'].map(risk_labels)
    
    # return buildings_with_risk
    pass

# Alternative: Use point-in-polygon for building centroids
def assign_risk_by_centroid(buildings_gdf, whp_raster_path):
    """Assign risk based on building centroid location"""
    # Get building centroids
    # centroids = buildings_gdf.copy()
    # centroids['geometry'] = centroids.geometry.centroid
    
    # Extract raster values at centroid points
    # centroid_stats = rasterstats.point_query(
    #     centroids,
    #     whp_raster_path,
    #     interpolate='nearest'
    # )
    
    # buildings_gdf['whp_risk_class'] = centroid_stats
    # return buildings_gdf
    pass

print("Building-level risk assignment methods:")
print("1. Majority rule: Use most common WHP value within building footprint")
print("2. Centroid method: Use WHP value at building center point")

print("\nExample code:")
print("# Method 1: Majority rule")
print("building_stats = rasterstats.zonal_stats(buildings, whp_raster, stats=['majority'])")
print("buildings['risk_class'] = [stat['majority'] for stat in building_stats]")
print("\n# Method 2: Centroid")
print("centroids = buildings.copy()")
print("centroids['geometry'] = centroids.geometry.centroid") 
print("risk_values = rasterstats.point_query(centroids, whp_raster)")
print("buildings['risk_class'] = risk_values")

---

## Module 5: Visualization & Sharing Results (15 minutes)

Let's create compelling, interactive maps to display our findings using `leafmap`.

### Interactive Mapping and Visualization

In [ ]:
# Create interactive maps with leafmap

# Initialize map centered on LA County
# m = leafmap.Map(center=[34.0522, -118.2437], zoom=9)

# 1. Choropleth mapping: Population exposure by census tract
def create_population_exposure_map():
    """Create choropleth map showing population exposure"""
    # m = leafmap.Map(center=[34.0522, -118.2437], zoom=9)
    
    # Add choropleth layer
    # m.add_gdf(
    #     la_census_tracts_exposure,
    #     column='pop_high_risk',
    #     scheme='quantiles',
    #     k=5,
    #     cmap='Reds',
    #     legend_title='Population at High Risk',
    #     layer_name='Population Exposure'
    # )
    
    # Add LA County boundary
    # m.add_gdf(la_county, style={'fillOpacity': 0, 'weight': 2, 'color': 'black'})
    
    # return m
    pass

# 2. Categorical mapping: Buildings colored by risk class
def create_building_risk_map():
    """Create map showing buildings colored by WHP risk class"""
    # m = leafmap.Map(center=[34.0522, -118.2437], zoom=10)
    
    # Define colors for risk categories
    # risk_colors = {
    #     'Very Low': '#2166ac',
    #     'Low': '#5aae61', 
    #     'Moderate': '#fee08b',
    #     'High': '#f46d43',
    #     'Very High': '#a50026'
    # }
    
    # Add buildings with categorical styling
    # m.add_gdf(
    #     buildings_with_risk,
    #     column='risk_category',
    #     categorical=True,
    #     legend_title='Wildfire Risk Class',
    #     layer_name='Building Risk'
    # )
    
    # return m
    pass

# 3. Combined raster and vector visualization
def create_combined_map():
    """Create map combining raster and vector layers"""
    # m = leafmap.Map(center=[34.0522, -118.2437], zoom=9)
    
    # Add WHP raster as background
    # m.add_raster(whp_clipped, colormap='RdYlBu_r', layer_name='Wildfire Hazard Potential')
    
    # Add census tracts with population exposure
    # m.add_gdf(la_census_tracts_exposure, column='pop_high_risk', layer_name='Population Exposure')
    
    # Add county boundary
    # m.add_gdf(la_county, style={'fillOpacity': 0, 'weight': 3, 'color': 'white'})
    
    # return m
    pass

print("Visualization functions created:")
print("1. create_population_exposure_map() - Choropleth of population at risk")
print("2. create_building_risk_map() - Buildings colored by risk class") 
print("3. create_combined_map() - Combined raster and vector layers")

print("\nExample usage:")
print("m1 = create_population_exposure_map()")
print("m1  # Display map")
print("m1.to_html('population_exposure_map.html')  # Export to HTML")

### 3D Visualization and Export Options

In [ ]:
# 3D visualization and export options

# Optional: 3D visualization with leafmap
def create_3d_visualization():
    """Create 3D visualization of wildfire risk"""
    # m = leafmap.Map(center=[34.0522, -118.2437], zoom=10)
    
    # Add 3D terrain layer
    # m.add_3d_terrain(
    #     source='https://elevation-tiles-prod.s3.amazonaws.com/terrarium/{z}/{x}/{y}.png',
    #     exaggeration=1.5
    # )
    
    # Add population data as 3D extrusion
    # m.add_gdf(
    #     la_census_tracts_exposure,
    #     column='pop_high_risk',
    #     extrude=True,
    #     layer_name='3D Population Risk'
    # )
    
    # return m
    pass

# Export maps to various formats
def export_maps():
    """Export maps to different formats"""
    
    # Export to HTML (interactive)
    # m.to_html('wildfire_exposure_map.html')
    
    # Export to PNG (static image)
    # m.to_image('wildfire_exposure_map.png', width=1200, height=800)
    
    # Export data to various formats
    # la_census_tracts_exposure.to_file('results/population_exposure.gpkg', layer='population')
    # buildings_with_risk.to_file('results/building_risk.gpkg', layer='buildings')
    
    # Export summary statistics
    # summary_stats = {
    #     'total_population': la_census_tracts_exposure['POPULATION'].sum(),
    #     'population_at_risk': la_census_tracts_exposure['pop_high_risk'].sum(),
    #     'total_buildings': len(buildings_with_risk),
    #     'high_risk_buildings': len(buildings_with_risk[buildings_with_risk['whp_risk_class'] >= 4])
    # }
    
    # with open('results/summary_statistics.json', 'w') as f:
    #     json.dump(summary_stats, f, indent=2)
    
    print("Export options:")
    print("- Interactive HTML maps")
    print("- Static PNG images") 
    print("- GeoPackage files with results")
    print("- Summary statistics")

print("Advanced visualization options:")
print("1. create_3d_visualization() - 3D terrain with population extrusion")
print("2. export_maps() - Export to multiple formats")

print("\nFinal steps:")
print("# Create and display final map")
print("final_map = create_combined_map()")
print("final_map")
print("# Export results")
print("export_maps()")

---

## Workshop Summary

### What We've Accomplished

By the end of this workshop, you have:

1. **Set up a geospatial Python environment** with essential libraries (GeoPandas, Rasterio, Rasterstats, Leafmap)

2. **Mastered vector data operations** including:
   - Loading data from multiple sources (files, databases, APIs)
   - Understanding GeoDataFrame structure
   - Performing attribute-based and spatial selections
   - Exporting cleaned data

3. **Processed raster data** including:
   - Reading and understanding raster metadata
   - Reprojecting and clipping raster data
   - Preparing data for spatial analysis

4. **Conducted spatial analysis** to answer our research question:
   - Calculated population exposure to wildfire risk
   - Determined building exposure percentages
   - Assigned risk classes to individual buildings

5. **Created compelling visualizations** including:
   - Interactive choropleth maps
   - Categorical mapping by risk class
   - Combined raster-vector visualizations
   - 3D terrain visualizations
   - Export options for sharing results

### Key Research Findings

**Research Question:** How much of the population and how many buildings are exposed to wildfire risk in Los Angeles County?

**Results Summary:**
- Total population analyzed: [X] people
- Population at high wildfire risk: [Y] people ([Z]%)
- Total buildings analyzed: [A] buildings  
- Buildings in high-risk areas: [B] buildings ([C]%)

### Next Steps

- Apply these techniques to other research questions
- Explore additional geospatial libraries (e.g., OSMnx, Folium, Plotly)
- Integrate with machine learning workflows
- Scale analysis to larger geographic areas
- Automate workflows with Python scripts

### Resources for Continued Learning

- **Documentation:** GeoPandas, Rasterio, Leafmap official docs
- **Tutorials:** Geocomputation with Python, Python for Geographic Data Analysis
- **Communities:** PyData, OSGeo, Stack Overflow
- **Datasets:** OpenStreetMap, USGS, NASA, local government data portals

**Thank you for participating in this hands-on GIS workshop!**